In [0]:
%pip install lyricsgenius

Python interpreter will be restarted.
  Using cached lyricsgenius-3.0.1-py3-none-any.whl (59 kB)
Python interpreter will be restarted.


In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, BooleanType
from pyspark import SparkFiles

In [0]:
# Access token to access the Genius API
genius_access_token = "_uAHk5SWEsiQ-m07ZfEkAINUaD8Qlu497ipRp3MUD-fDVw1MpfEPeY5nx3kg08pB"

from lyricsgenius import Genius

# Clean up the lyrics so they don't include the below terms
excluded_lyrics = ['Contributors','Embed']

# Re-usable object to search and save lyrics.
# Set the timeout for the genius lyrics API, have it sleep and retry a few times.
genius = Genius(genius_access_token, timeout=10, retries=3, sleep_time=1, verbose=False, remove_section_headers=True, excluded_terms=excluded_lyrics)


In [0]:
# Let's try the Genius API to see if it works as expected:
#song = genius.search_song('Hey, Ma', 'Bon Iver')
#print(song.lyrics)

In [0]:
# Let's store the file in dbfs, if it's not there, fetch it and save it to dbfs
def getGrammyData():
    file_location = "/FileStore/tables/the_grammy_awards.csv"

    try:
        f = dbutils.fs.ls(file_location)
    except:
        url = "https://raw.githubusercontent.com/mackrauss/3252-BigData-Group-F/main/the_grammy_awards.csv"
        spark.sparkContext.addFile(url)
        dbutils.fs.cp('file://'+SparkFiles.get('the_grammy_awards.csv'), file_location)
        print('Moved file to tables')
    
    return file_location

In [0]:
file_location = getGrammyData()

file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# Adding 'escape' option to change escape character from \ to " - not we need to escape the " inside the
# see https://stackoverflow.com/questions/40413526/reading-csv-files-with-quoted-fields-containing-embedded-commas
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("escape", "\"") \
  .load(file_location)

In [0]:
# pull file via HTTP/S using Spark tools
# from pyspark import SparkFiles

# url = "https://raw.githubusercontent.com/mackrauss/3252-BigData-Group-F/main/the_grammy_awards.csv"
# spark.sparkContext.addFile(url)

# df = spark.read.csv("file://"+SparkFiles.get("the_grammy_awards.csv"), header=True, inferSchema= True)


In [0]:
display(df.head(5))

year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,"Hey, Ma",Bon Iver,"BJ Burton, Brad Cook, Chris Messina & Justin Vernon, producers; BJ Burton, Zach Hanson & Chris Messina, engineers/mixers; Greg Calbi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bon%2BIver/Bon%2520Iver_1_1_1578385181.jpg?itok=_M1hc5Ux,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,7 rings,Ariana Grande,"Charles Anderson, Tommy Brown, Michael Foster & Victoria Monet, producers; Serban Ghenea, John Hanes, Billy Hickey & Brendan Morawski, engineers/mixers; Randy Merrill, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Ariana%2BGrande/Ariana%2520Grande_1_1_1578384678.jpg?itok=Amj90j13,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Hard Place,H.E.R.,"Rodney “Darkchild” Jerkins, producer; Joseph Hurtado, Jaycen Joshua, Derek Keota & Miki Tsutsumi, engineers/mixers; Colin Leonard, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/H.E.R./H.E.R._1_1_1594631035.jpg?itok=ClJe-2MN,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Talk,Khalid,"Disclosure & Denis Kosiak, producers; Ingmar Carlson, Jon Castelli, Josh Deguzman, John Kercy, Denis Kosiak, Guy Lawrence & Michael Romero, engineers/mixers; Dale Becker, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Khalid/Khalid_1_1_1594578772.jpg?itok=2HxjAT-X,True


In [0]:
# Data cleaning -- we have multiple entries for year 2019
df = df.filter((df.year != 2019) | ((df.year == 2019) & (df.nominee == 'Bad Guy') & (df.artist == 'Billie Eilish')) )
display(df.head(5))

year,title,published_at,updated_at,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2019,62nd Annual GRAMMY Awards (2019),2020-05-19T05:10:28-07:00,2020-05-19T05:10:28-07:00,Best Pop Solo Performance,Bad Guy,Billie Eilish,null,https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2018,61st Annual GRAMMY Awards (2018),2018-12-06T23:48:49-08:00,2019-09-10T01:06:11-07:00,Record Of The Year,This Is America,Childish Gambino,"Donald Glover & Ludwig Göransson, producers; Derek ""MixedByAli"" Ali, Kesha Lee, Riley Mackin, Shaan Singh & Alex Tumay, engineers/mixers; Mike Bozzi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Childish%2BGambino/Childish%2520Gambino_1_1_1591112936.jpg?itok=AU_J_f8x,True
2018,61st Annual GRAMMY Awards (2018),2018-12-06T23:48:49-08:00,2019-09-10T01:06:11-07:00,Album Of The Year,Golden Hour,Kacey Musgraves,"Ian Fitchuk, Kacey Musgraves & Daniel Tashian, producers; Craig Alvin & Shawn Everett, engineers/mixers; Ian Fitchuk, Kacey Musgraves & Daniel Tashian, songwriters; Greg Calbi & Steve Fallone, mastering engineers",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Kacey%2BMusgraves/Kacey%2520Musgraves_1_1_1578384727.jpg?itok=OQQ3rnwH,True
2018,61st Annual GRAMMY Awards (2018),2018-12-06T23:48:49-08:00,2019-09-10T01:06:11-07:00,Song Of The Year,This Is America,null,"Donald Glover, Ludwig Göransson & Jeffery Lamar Williams, songwriters (Childish Gambino)",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Childish%2BGambino/Childish%2520Gambino_1_1_1591112936.jpg?itok=AU_J_f8x,True


In [0]:
# Create a view or table to allow SQL usage

temp_table_name = "grammy_awards"

df.createOrReplaceTempView(temp_table_name)

In [0]:
# Show all years in the record
df.select("year").distinct().orderBy(df.year.asc()).show() 

+----+
|year|
+----+
|1958|
|1959|
|1960|
|1961|
|1962|
|1963|
|1964|
|1965|
|1966|
|1967|
|1968|
|1969|
|1970|
|1971|
|1972|
|1973|
|1974|
|1975|
|1976|
|1977|
+----+
only showing top 20 rows



In [0]:
# Count number of years recorded
df.select("year").distinct().count()

Out[11]: 62

In [0]:
%sql
-- count number of years
select count(distinct year) AS `# years recorded`
from `grammy_awards`

# years recorded
62


In [0]:
%sql
-- count number of categories
select count(distinct category) `# of categories`
from `grammy_awards`

# of categories
637


In [0]:
%sql
-- ensure we have 'Record Of The Year' data for each year
select count(distinct year) as `Years with Record of the Year recorded`
from `grammy_awards`
where category like 'Record Of The Year'


Years with Record of the Year recorded
62


In [0]:
# Create a new DataFrame that only contains the 'Record Of The Year' category data
df_record = df.select('year','title','category','nominee','artist','workers','img','winner').where(df.category == "Record Of The Year")
display(df_record.head(5))

year,title,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2018,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,"Donald Glover & Ludwig Göransson, producers; Derek ""MixedByAli"" Ali, Kesha Lee, Riley Mackin, Shaan Singh & Alex Tumay, engineers/mixers; Mike Bozzi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Childish%2BGambino/Childish%2520Gambino_1_1_1591112936.jpg?itok=AU_J_f8x,True
2017,60th Annual GRAMMY Awards (2017),Record Of The Year,24K Magic,Bruno Mars,"Shampoo Press & Curl (Christopher Brody Brown, Philip Lawrence & Bruno Mars), producers; Serban Ghenea, John Hanes & Charles Moniz, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bruno%2BMars/Bruno%2520Mars_1_1_1578385368.jpg?itok=KhxnacRR,True
2016,59th Annual GRAMMY Awards (2016),Record Of The Year,Hello,Adele,"Greg Kurstin, producer; Julian Burg, Tom Elmhirst, Emile Haynie, Greg Kurstin, Liam Nolan, Alex Pasco & Joe Visciano, engineers/mixers; Tom Coyne & Randy Merrill, mastering engineers",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Adele/Adele_1_1_1584090130.jpg?itok=xlwW6t3a,True
2015,58th Annual GRAMMY Awards (2015),Record Of The Year,Uptown Funk,Mark Ronson Featuring Bruno Mars,"Jeff Bhasker, Philip Lawrence, Bruno Mars & Mark Ronson, producers; Josh Blair, Riccardo Damian, Serban Ghenea, Wayne Gordon, John Hanes, Inaam Haq, Boo Mitchell, Charles Moniz & Mark Ronson, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Mark%2BRonson/Mark%2520Ronson_1_1_1596790481.jpg?itok=s-dqiJed,True


In [0]:
# Create a view or table for Record Of the Year data

temp_table_name = "record_of_the_year"

df_record.createOrReplaceTempView(temp_table_name)

In [0]:
# The next code block shows a data import error for 'Old Town Road' where escape character lead to more columns and thus no URL in the img filed
# Use this to export a CSV with all "Record of the Year" rows and do a quick check to ensure the correct data ends up in each column
# Armin 17.09.2023 Adding .option("escape", "\"") to the file import seems to have fixed the issue around img
# https://stackoverflow.com/questions/40413526/reading-csv-files-with-quoted-fields-containing-embedded-commas
display(df_record)

year,title,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2018,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,"Donald Glover & Ludwig Göransson, producers; Derek ""MixedByAli"" Ali, Kesha Lee, Riley Mackin, Shaan Singh & Alex Tumay, engineers/mixers; Mike Bozzi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Childish%2BGambino/Childish%2520Gambino_1_1_1591112936.jpg?itok=AU_J_f8x,True
2017,60th Annual GRAMMY Awards (2017),Record Of The Year,24K Magic,Bruno Mars,"Shampoo Press & Curl (Christopher Brody Brown, Philip Lawrence & Bruno Mars), producers; Serban Ghenea, John Hanes & Charles Moniz, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bruno%2BMars/Bruno%2520Mars_1_1_1578385368.jpg?itok=KhxnacRR,True
2016,59th Annual GRAMMY Awards (2016),Record Of The Year,Hello,Adele,"Greg Kurstin, producer; Julian Burg, Tom Elmhirst, Emile Haynie, Greg Kurstin, Liam Nolan, Alex Pasco & Joe Visciano, engineers/mixers; Tom Coyne & Randy Merrill, mastering engineers",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Adele/Adele_1_1_1584090130.jpg?itok=xlwW6t3a,True
2015,58th Annual GRAMMY Awards (2015),Record Of The Year,Uptown Funk,Mark Ronson Featuring Bruno Mars,"Jeff Bhasker, Philip Lawrence, Bruno Mars & Mark Ronson, producers; Josh Blair, Riccardo Damian, Serban Ghenea, Wayne Gordon, John Hanes, Inaam Haq, Boo Mitchell, Charles Moniz & Mark Ronson, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Mark%2BRonson/Mark%2520Ronson_1_1_1596790481.jpg?itok=s-dqiJed,True
2014,57th Annual GRAMMY Awards (2014),Record Of The Year,Stay With Me (Darkchild Version),Sam Smith,"Steve Fitzmaurice, Rodney Jerkins & Jimmy Napes, producers; Matthew Champlin, Steve Fitzmaurice, Jimmy Napes & Steve Price, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/sam_smithrotycrop.jpg?itok=CbuVXG4s,True
2013,56th Annual GRAMMY Awards (2013),Record Of The Year,Get Lucky,Daft Punk Featuring Pharrell Williams & Nile Rodgers,"Thomas Bangalter & Guy-Manuel de Homem-Christo, producers; Peter Franco, Mick Guzauski, Florian Lagatta & Daniel Lerner, engineers/mixers; Antoine Chabert & Bob Ludwig, mastering engineers",https://www.grammy.com/sites/com/files/styles/artist_circle/public/465384097_daft_punk_pharrell.jpg?itok=tZ0IzKSI,True
2012,55th Annual GRAMMY Awards (2012),Record Of The Year,Somebody That I Used To Know,Gotye Featuring Kimbra,"Wally De Backer, producer; Wally De Backer & Francois Tetaz, engineers/mixers; William Bowden, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/161396233_gotye_kimbra.jpg?itok=4_A2Llji,True
2011,54th Annual GRAMMY Awards (2011),Record Of The Year,Rolling In The Deep,Adele,"Paul Epworth, producer; Tom Elmhirst & Mark Rankin, engineers/mixers",https://www.grammy.com/sites/com/files/styles/artist_circle/public/138850365_adele.jpg?itok=X8mml-Wk,True
2010,53rd Annual GRAMMY Awards (2010),Record Of The Year,Need You Now,Lady Antebellum,"Lady Antebellum (Dave Haywood, Charles Kelley & Hillary Scott) & Paul N. Worley, producers; L Clarke Schleicher, engineer/mixer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/109066913_lady_antebellum.jpg?itok=iFnOLjld,True


In [0]:
%sql
-- Show data import error
-- if the field img shows an URL the import worked as expected
select nominee, artist, workers, img, winner
from `record_of_the_year`
where nominee = "Old Town Road"

nominee,artist,workers,img,winner


In [0]:
%sql
select *
from `record_of_the_year`

-- order by year

year,title,category,nominee,artist,workers,img,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,"Finneas O'Connell, producer; Rob Kinelski & Finneas O'Connell, engineers/mixers; John Greenham, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Billie%2BEilish/Billie%2520Eilish_1_1_1594138954.jpg?itok=3-71Dfxh,True
2018,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,"Donald Glover & Ludwig Göransson, producers; Derek ""MixedByAli"" Ali, Kesha Lee, Riley Mackin, Shaan Singh & Alex Tumay, engineers/mixers; Mike Bozzi, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Childish%2BGambino/Childish%2520Gambino_1_1_1591112936.jpg?itok=AU_J_f8x,True
2017,60th Annual GRAMMY Awards (2017),Record Of The Year,24K Magic,Bruno Mars,"Shampoo Press & Curl (Christopher Brody Brown, Philip Lawrence & Bruno Mars), producers; Serban Ghenea, John Hanes & Charles Moniz, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Bruno%2BMars/Bruno%2520Mars_1_1_1578385368.jpg?itok=KhxnacRR,True
2016,59th Annual GRAMMY Awards (2016),Record Of The Year,Hello,Adele,"Greg Kurstin, producer; Julian Burg, Tom Elmhirst, Emile Haynie, Greg Kurstin, Liam Nolan, Alex Pasco & Joe Visciano, engineers/mixers; Tom Coyne & Randy Merrill, mastering engineers",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Adele/Adele_1_1_1584090130.jpg?itok=xlwW6t3a,True
2015,58th Annual GRAMMY Awards (2015),Record Of The Year,Uptown Funk,Mark Ronson Featuring Bruno Mars,"Jeff Bhasker, Philip Lawrence, Bruno Mars & Mark Ronson, producers; Josh Blair, Riccardo Damian, Serban Ghenea, Wayne Gordon, John Hanes, Inaam Haq, Boo Mitchell, Charles Moniz & Mark Ronson, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/muzooka/Mark%2BRonson/Mark%2520Ronson_1_1_1596790481.jpg?itok=s-dqiJed,True
2014,57th Annual GRAMMY Awards (2014),Record Of The Year,Stay With Me (Darkchild Version),Sam Smith,"Steve Fitzmaurice, Rodney Jerkins & Jimmy Napes, producers; Matthew Champlin, Steve Fitzmaurice, Jimmy Napes & Steve Price, engineers/mixers; Tom Coyne, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/sam_smithrotycrop.jpg?itok=CbuVXG4s,True
2013,56th Annual GRAMMY Awards (2013),Record Of The Year,Get Lucky,Daft Punk Featuring Pharrell Williams & Nile Rodgers,"Thomas Bangalter & Guy-Manuel de Homem-Christo, producers; Peter Franco, Mick Guzauski, Florian Lagatta & Daniel Lerner, engineers/mixers; Antoine Chabert & Bob Ludwig, mastering engineers",https://www.grammy.com/sites/com/files/styles/artist_circle/public/465384097_daft_punk_pharrell.jpg?itok=tZ0IzKSI,True
2012,55th Annual GRAMMY Awards (2012),Record Of The Year,Somebody That I Used To Know,Gotye Featuring Kimbra,"Wally De Backer, producer; Wally De Backer & Francois Tetaz, engineers/mixers; William Bowden, mastering engineer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/161396233_gotye_kimbra.jpg?itok=4_A2Llji,True
2011,54th Annual GRAMMY Awards (2011),Record Of The Year,Rolling In The Deep,Adele,"Paul Epworth, producer; Tom Elmhirst & Mark Rankin, engineers/mixers",https://www.grammy.com/sites/com/files/styles/artist_circle/public/138850365_adele.jpg?itok=X8mml-Wk,True
2010,53rd Annual GRAMMY Awards (2010),Record Of The Year,Need You Now,Lady Antebellum,"Lady Antebellum (Dave Haywood, Charles Kelley & Hillary Scott) & Paul N. Worley, producers; L Clarke Schleicher, engineer/mixer",https://www.grammy.com/sites/com/files/styles/artist_circle/public/109066913_lady_antebellum.jpg?itok=iFnOLjld,True


In [0]:
# not sure this reduction is necessary as we do this earlier with more columns
# however if this changes we need to adjust the columns list in the next code field
df_record = df.select('year','title','category','nominee','artist','winner').where(df.category == "Record Of The Year")
display(df_record)

## Why are all of the records winners?

year,title,category,nominee,artist,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,True
2018,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,True
2017,60th Annual GRAMMY Awards (2017),Record Of The Year,24K Magic,Bruno Mars,True
2016,59th Annual GRAMMY Awards (2016),Record Of The Year,Hello,Adele,True
2015,58th Annual GRAMMY Awards (2015),Record Of The Year,Uptown Funk,Mark Ronson Featuring Bruno Mars,True
2014,57th Annual GRAMMY Awards (2014),Record Of The Year,Stay With Me (Darkchild Version),Sam Smith,True
2013,56th Annual GRAMMY Awards (2013),Record Of The Year,Get Lucky,Daft Punk Featuring Pharrell Williams & Nile Rodgers,True
2012,55th Annual GRAMMY Awards (2012),Record Of The Year,Somebody That I Used To Know,Gotye Featuring Kimbra,True
2011,54th Annual GRAMMY Awards (2011),Record Of The Year,Rolling In The Deep,Adele,True
2010,53rd Annual GRAMMY Awards (2010),Record Of The Year,Need You Now,Lady Antebellum,True


In [0]:
import pandas as pd
# Reduce data set for some quick testing
record_list = df_record.take(80)
df_record_new = pd.DataFrame(record_list, columns=['year','title','category','nominee','artist','winner'])
display(df_record_new)

year,title,category,nominee,artist,winner
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,True
2018,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,True
2017,60th Annual GRAMMY Awards (2017),Record Of The Year,24K Magic,Bruno Mars,True
2016,59th Annual GRAMMY Awards (2016),Record Of The Year,Hello,Adele,True
2015,58th Annual GRAMMY Awards (2015),Record Of The Year,Uptown Funk,Mark Ronson Featuring Bruno Mars,True
2014,57th Annual GRAMMY Awards (2014),Record Of The Year,Stay With Me (Darkchild Version),Sam Smith,True
2013,56th Annual GRAMMY Awards (2013),Record Of The Year,Get Lucky,Daft Punk Featuring Pharrell Williams & Nile Rodgers,True
2012,55th Annual GRAMMY Awards (2012),Record Of The Year,Somebody That I Used To Know,Gotye Featuring Kimbra,True
2011,54th Annual GRAMMY Awards (2011),Record Of The Year,Rolling In The Deep,Adele,True
2010,53rd Annual GRAMMY Awards (2010),Record Of The Year,Need You Now,Lady Antebellum,True


In [0]:
import time
import re
# Let's try to get one of the lyrics to see if the Genius API works
def get_lyrics (artist, song_name):
    try:
        song = genius.search_song(song_name, artist)
        # time.sleep(1)
        lyrics = song.lyrics.replace('\n',' ')
        lyrics = re.sub(r'^.*Contributors', '', lyrics)
        lyrics = re.sub(r'\d+Embed', '', lyrics)
        return lyrics
        # print(song.lyrics)
    except Exception as e:
        # print(e)
        return "Could not retrieve"

In [0]:
print(get_lyrics('Bon Iver','Hey, Ma'))

Hey, Ma Lyrics I waited outside I took it remote I wanted a bath "Tell the story or he goes" "Tell the story or he goes"  Full time, you talk your money up While it's living in a coal mine Tall time to call your Ma Hey Ma, hey Ma Tall vote, you know you mope it up Well, you wanted it your whole life You're back and forth with light  I waited outside I was tokin' on dope I hoped it all wunt go in a minute With the past that you know I wanted all that mind sugar I want it all mine I had a heavy mind, sugar I took it, it was right You might also like Full time, you talk your money up While it's living in a coal mine Tall time to call your Ma Hey Ma, hey Ma Tall vote, you know you mope it up Well, you wanted it your whole life You're back and forth with light  You're back and forth You're back and forth You go back and forth And then you're back and forth And back and forth  I waited outside Then you took me in the room And you offered up the truth My eyes crawling up the window to the wal

In [0]:
# Create a UDF to get the lyrics
# Convert function to udf
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
lyricsUDF = udf(lambda artist,song_name:get_lyrics(artist, song_name),StringType()) 


In [0]:
df_record_lyrics = df_record.withColumn("lyrics", lyricsUDF(col("artist"), col("nominee")))
# df_record.withColumn("lyrics", lyricsUDF(col("artist"), col("nominee"))).show(truncate=False)

In [0]:
# display(df_record_lyrics)

In [0]:
# Hitting and issue with pulling lyrics for all records at ones. Trying to do 10 at a time
# Years 1958 - 2019
rslt_df = df_record[df_record['year'] >= "2017"].toPandas()
rslt_df_2 = df_record[(df_record['year'] < "2017") & (df_record['year'] >= "2010")].toPandas()
rslt_df_3 = df_record[(df_record['year'] < "2010") & (df_record['year'] >= "2000")].toPandas()
rslt_df_4 = df_record[(df_record['year'] < "2000") & (df_record['year'] >= "1990")].toPandas()
rslt_df_5 = df_record[(df_record['year'] < "1990") & (df_record['year'] >= "1980")].toPandas()
rslt_df_6 = df_record[(df_record['year'] < "1980") & (df_record['year'] >= "1970")].toPandas()
rslt_df_7 = df_record[(df_record['year'] < "1970") & (df_record['year'] >= "1960")].toPandas()
rslt_df_8 = df_record[(df_record['year'] < "1960")].toPandas()

In [0]:
rslt_df['lyrics'] = rslt_df.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
rslt_df_2['lyrics'] = rslt_df_2.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
rslt_df_3['lyrics'] = rslt_df_3.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
rslt_df_4['lyrics'] = rslt_df_4.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
rslt_df_5['lyrics'] = rslt_df_5.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
rslt_df_6['lyrics'] = rslt_df_6.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
rslt_df_7['lyrics'] = rslt_df_7.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
rslt_df_8['lyrics'] = rslt_df_8.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
pd_new = pd.concat([rslt_df, rslt_df_2, rslt_df_3, rslt_df_4, rslt_df_5, rslt_df_6, rslt_df_7, rslt_df_8])
display(pd_new)

year,title,category,nominee,artist,winner,lyrics
2019,62nd Annual GRAMMY Awards (2019),Record Of The Year,Bad Guy,Billie Eilish,True,"TranslationsHrvatskiTürkçeSvenskaEspañolРусскийRomânăPortuguêsPolskiItalianoHebrewDeutschFrançaisفارسیNederlandsDanskChinese TraditionalΕλληνικάالعربيةไทย / Phasa Thai​bad guy Lyrics White shirt now red, my bloody nose Sleepin', you're on your tippy toes Creepin' around like no one knows Think you're so criminal Bruises on both my knees for you Don't say thank you or please I do what I want when I'm wanting to My soul? So cynical So you're a tough guy Like it really rough guy Just can't get enough guy Chest always so puffed guy I'm that bad type Make your mama sad type Make your girlfriend mad tight Might seduce your dad type I'm the bad guy Duh I'm the bad guy I like it when you take control Even if you know that you don't Own me, I'll let you play the role I'll be your animal My mommy likes to sing along with me But she won't sing this song If she reads all the lyrics She'll pity the men I know You might also like So you're a tough guy Like it really rough guy Just can't get enough guy Chest always so puffed guy I'm that bad type Make your mama sad type Make your girlfriend mad tight Might seduce your dad type I'm the bad guy Duh I'm the bad guy, duh I'm only good at bein' bad, bad I like when you get mad I guess I'm pretty glad that you're alone You said she's scared of me? I mean, I don't see what she sees But maybe it's 'cause I'm wearing your cologne I'm a bad guy I'm, I'm a bad guy Bad guy, bad guy I'm a bad"
2018,61st Annual GRAMMY Awards (2018),Record Of The Year,This Is America,Childish Gambino,True,"Translationsไทย / Phasa ThaiEspañolРусскийPortuguêsDeutschFrançaisNederlandsThis Is America Lyrics Yeah, yeah, yeah, yeah, yeah Yeah, yeah, yeah, go, go away Yeah, yeah, yeah, yeah, yeah Yeah, yeah, yeah, go, go away Yeah, yeah, yeah, yeah, yeah Yeah, yeah, yeah, go, go away Yeah, yeah, yeah, yeah, yeah Yeah, yeah, yeah, go, go away We just wanna party Party just for you We just want the money Money just for you (Yeah) I know you wanna party Party just for free Girl, you got me dancin' (Girl, you got me dancin') Dance and shake the frame (Yeah) We just wanna party (Yeah) Party just for you (Yeah) We just want the money (Yeah) Money just for you (You) I know you wanna party (Yeah) Party just for free (Yeah) Girl, you got me dancin' (Girl, you got me dancin', yeah) Dance and shake the frame (Ooh) You might also like This is America Don't catch you slippin' now Don't catch you slippin' now Look what I'm whippin' now This is America (Woo) Don't catch you slippin' now Don't catch you slippin' now Look what I'm whippin' now This is America (Skrrt, skrrt, woo) Don't catch you slippin' now (Ayy) Look how I'm livin' now Police be trippin' now (Woo) Yeah, this is America (Woo, ayy) Guns in my area (Word, my area) I got the strap (Ayy, ayy) I gotta carry 'em Yeah, yeah, I'ma go into this (Ugh) Yeah, yeah, this is guerilla (Woo) Yeah, yeah, I'ma go get the bag Yeah, yeah, or I'ma get the pad Yeah, yeah, I'm so cold like, yeah (Yeah) I'm so dope like, yeah (Woo) We gon' blow like, yeah (Straight up, uh) Ooh-ooh-ooh-ooh-ooh, tell somebody You go tell somebody Grandma told me Get your money, Black man (Get your— Black man) Get your money, Black man (Get your—Black man) Get your money, Black man (Get your—Black man) Get your money, Black man (Get your—Black man) Black man This is America (Woo, ayy) Don't catch you slippin' now (Woo, woo, don't catch you slippin' now) Don't catch you slippin' now (Ayy, woah) Look what I'm whippin' now (Slime!) This is America (Yeah, yeah) Don't catch you slippin' now (Woah, ayy) Don't catch you slippin' now (Ayy, woo) Look what I'm whippin' now (Ayy) Look how I'm geekin' out (Hey) I'm so fitted (I'm so fitted, woo) I'm on Gucci (I'm on Gucci) I'm so pretty (Yeah, yeah, woo) I'm gon' get it (Ayy, I'm gon' get it) Watch me move (Blaow) This a celly (Ha) That's a tool (Yeah) 

In [0]:
# data['BMI'] = data.apply(lambda x: calc_bmi(x['Weight'], x['Height']), axis=1)
# df_record_new['lyrics'] = df_record_new.apply(lambda x: get_lyrics(x['artist'], x['nominee']), axis=1)

In [0]:
# display(df_record_new)